In [6]:
import io

import netaddr
import ipaddress

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_stats, read_delegated_extended_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df_extended = read_delegated_extended_stats(io.StringIO(await resp.text()))

In [3]:
df_orig = df_extended.copy()

In [8]:
def int_to_128bit_bytes(inp: int) -> bytes:
    buf = ipaddress.ip_address(inp).packed
    to_pad = len(buf) - (128//8)

    return b'\x00' * to_pad + buf        

df_extended = df_extended[df_extended.afi != "asn"].copy()
df_extended['resource_first'] = df_extended.resource.map(lambda x: int_to_128bit_bytes(x.first))
df_extended['resource_last'] = df_extended.resource.map(lambda x: int_to_128bit_bytes(x.last))

df_extended['resource'] = df_extended.resource.map(str)
del df_extended['raw_resource']

In [10]:
df_extended.to_parquet('nro-delegated-stats-noas-boundaries-20241105.parquet', index=False)

In [9]:
df_extended.sample(5)

,rir,country,afi,length,date,status,opaque_id,category,resource,resource_first,resource_last
563186,lacnic,ZZ,ipv6,29,2024-11-05 00:00:00+00:00,reserved,lacnic,e-stats,2800:978::/29,b'(\x00\tx\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'(\x00\t\x7f\xff\xff\xff\xff\xff\xff\xff\xff\...
681504,ripencc,ZZ,ipv6,31,2024-11-05 00:00:00+00:00,reserved,ripencc,e-stats,2a0c:6fc2::/31,b'*\x0co\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x...,b'*\x0co\xc3\xff\xff\xff\xff\xff\xff\xff\xff\x...
660289,ripencc,ZZ,ipv6,29,2024-11-05 00:00:00+00:00,reserved,ripencc,e-stats,2a06:8bc8::/29,b'*\x06\x8b\xc8\x00\x00\x00\x00\x00\x00\x00\x0...,b'*\x06\x8b\xcf\xff\xff\xff\xff\xff\xff\xff\xf...
501357,arin,ZZ,ipv6,27,2024-11-05 00:00:00+00:00,available,arin,e-stats,2604:4820::/27,b'&\x04H \x00\x00\x00\x00\x00\x00\x00\x00\x00\...,b'&\x04H?\xff\xff\xff\xff\xff\xff\xff\xff\xff\...
613844,lacnic,ZZ,ipv6,32,2024-11-05 00:00:00+00:00,available,lacnic,e-stats,2804:7e29::/32,b'(\x04~)\x00\x00\x00\x00\x00\x00\x00\x00\x00\...,b'(\x04~)\xff\xff\xff\xff\xff\xff\xff\xff\xff\...
